### Import Gurobi

In [1]:
# Import gurobipy
from gurobipy import *

### Create Model, Decision Variables & Objective Function

In [2]:
# Create a model
m = Model("Vehicle Production Optimization")

Academic license - for non-commercial use only - expires 2021-03-14
Using license file /Users/jessietam/gurobi.lic


In [3]:
# Create the decision variables
S = m.addVar(vtype=GRB.CONTINUOUS, name="Subcompact Car")
C = m.addVar(vtype=GRB.CONTINUOUS, name="Compact Car")
I = m.addVar(vtype=GRB.CONTINUOUS, name="Intermediate Car")
L = m.addVar(vtype=GRB.CONTINUOUS, name="Luxury Car")
T = m.addVar(vtype=GRB.CONTINUOUS, name="Truck")
V = m.addVar(vtype=GRB.CONTINUOUS, name="Van")


# Define objective function - maximize profit
m.setObjective(150*S + 225*C + 250*I + 500*L + 400*T + 200*V, GRB.MAXIMIZE)

### Define Constraints

In [4]:
# Market potential contraints for each vehicle
m.addConstr(S <= 600000, "Subcompact Market Potential")
m.addConstr(C <= 400000, "Compact Market Potential")
m.addConstr(I <= 300000, "Intermediate Market Potential")
m.addConstr(L <= 225000, "Luxury Market Potential")
m.addConstr(T <= 325000, "Truck Market Potential")
m.addConstr(V <= 100000, "Van Market Potential")

# Vehicle production limits
m.addConstr(S + C + I + L + T + V <= 1200000, "Vendor Production Capacity")
m.addConstr(S + C <= 620000, "S & C Facility Capacity")
m.addConstr(I + L <= 400000, "I & L Facility Capacity")
m.addConstr(T + V <= 275000, "T & V Facility Capacity")

# Marketing strategy constraint
m.addConstr(0.5*S + 0.5*C - 0.5*I - 0.5*L >= 0, "At least 50% of cars are S & C")

# Average fuel economy is at least 27 mpg
m.addConstr(13*S + 7*C - 12*I - 15*L - 7*T - 2*V >= 0, "Average Fuel Economy Standard")

# Optimize model
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 12 rows, 6 columns and 28 nonzeros
Model fingerprint: 0x4c437ced
Coefficient statistics:
  Matrix range     [5e-01, 2e+01]
  Objective range  [2e+02, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 1e+06]
Presolve removed 6 rows and 0 columns
Presolve time: 0.02s
Presolved: 6 rows, 6 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+08   2.050000e+06   0.000000e+00      0s
       6    3.5800000e+08   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.06 seconds
Optimal objective  3.580000000e+08


In [5]:
# Print results
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))
    
# Print results for objective
print('Obj: %g' % m.objVal)

Subcompact Car 320000
Compact Car 300000
Intermediate Car 80000
Luxury Car 225000
Truck 275000
Van 0
Obj: 3.58e+08


In [6]:
# Display binding constraints
for c in m.getConstrs():
    print('%s: %g' % (c.ConstrName, c.slack))

Subcompact Market Potential: 280000
Compact Market Potential: 100000
Intermediate Market Potential: 220000
Luxury Market Potential: 0
Truck Market Potential: 50000
Van Market Potential: 100000
Vendor Production Capacity: 0
S & C Facility Capacity: 0
I & L Facility Capacity: 95000
T & V Facility Capacity: 0
At least 50% of cars are S & C: -157500
Average Fuel Economy Standard: 0


In [7]:
# Create variable for shadow prices
shadow_prices = m.getAttr('Pi') 

# Print non-zero shadow prices
m.printAttr('Pi')


  Constraint           Pi 
-------------------------
Luxury Market Potential        212.5 
Vendor Production Capacity          100 
S & C Facility Capacity        212.5 
T & V Facility Capacity        212.5 
Average Fuel Economy Standard        -12.5 


In [8]:
# Print reduced cost
for v in m.getVars():
    print('%s: %g' % (v.varName, v.rc))

Subcompact Car: 0
Compact Car: 0
Intermediate Car: 0
Luxury Car: 0
Truck: 0
Van: -137.5
